In [1]:
server_args = []
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

from manipulation import running_as_notebook


# Imports
import numpy as np
import pydot
from ipywidgets import Dropdown, Layout
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, autoDiffToGradientMatrix, SnoptSolver, IpoptSolver,
    initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix,
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint,
    MeshcatContactVisualizer
)

import pickle

In [2]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)

In [3]:
#add table
ground = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/ground.urdf"), "ground")

#add box
cracker_box = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/003_cracker_box_test.urdf"), "cracker_box")


In [4]:
plant.Finalize()
plant.set_name("plant")

In [5]:
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())


draw_frames = True
frames_to_draw = [cracker_box_id] if draw_frames else []
meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, frames_to_draw=frames_to_draw, axis_length=0.15,
                 axis_radius=0.001,)

#visualizer the contact
contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
                                       force_threshold=0.0,
                                       contact_force_scale=0.05, 
                                       contact_force_radius=0.001,   
                                       plant=plant))
builder.Connect(plant.get_contact_results_output_port(),
                contact_visualizer.GetInputPort("contact_results"))


diagram = builder.Build()
# context = diagram.CreateDefaultContext()


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


In [27]:
print(q0_o)
print(v0_o)
# 1.,        0.,        0.,        0.,        0.,       0.,        0.03340011,
#                                 0.,  0.,  0.,  0.5, 0.,  0.

[1.        0.        0.        0.        0.        0.        0.0334001]
[0.  0.  0.  0.5 0.  0. ]


In [67]:
# Set up a simulator to run this diagram
simulator = Simulator(diagram)

# Set the initial conditions
context = simulator.get_mutable_context()
plant_context = plant.GetMyContextFromRoot(context)
X_WB = RigidTransform(RotationMatrix(), [0., 0., 0.0334 + 1e-07])
plant.SetFreeBodyPose(plant_context, 
                              plant.GetBodyByName("base_link_cracker", cracker_box),
                              X_WB)
q0_o = plant.GetPositions(plant_context, cracker_box)
v0_o = plant.GetVelocities(plant_context, cracker_box)
v0_o[3] = 0.5

plant.SetPositions(plant_context, cracker_box, q0_o)
plant.SetVelocities(plant_context, cracker_box, v0_o)


meshcat.load()
diagram.Publish(context)

In [68]:
context.SetTime(0.0)
simulator.set_target_realtime_rate(1.0)

N = 100
v_last = np.zeros((6))
for n in range(N):
    sim_status = simulator.AdvanceTo(1.0*n/N)
    
    print("sim_status", n, ":", sim_status.FormatMessage())
    q = plant.GetPositions(plant_context, cracker_box)
    v = plant.GetVelocities(plant_context, cracker_box)
    vdot = (v - v_last)/(1/N)
    v_last = v
    print("q", n, ":", q)
    print("v", n, ":", v)
#     print("vdot", n, ":", vdot)
#     body_spatial_acc = plant.get_body_spatial_accelerations_output_port().Eval(plant_context)
#     print("body_spatial_acc", n, ":", body_spatial_acc[2].rotational(), body_spatial_acc[2].translational())
    generalized_acc = plant.get_generalized_acceleration_output_port().Eval(plant_context)
    print("generalized_acc", n, ":", generalized_acc)
    print("\n")
   
    
#     results = plant.get_contact_results_output_port().Eval(plant_context)
#     for i in range(results.num_point_pair_contacts()):
#         info = results.point_pair_contact_info(i)
#         pair = info.point_pair()
#         print("depth:",pair.depth,"  id_A:", pair.id_A,"  id_B:", pair.id_B,"  nhat_BA_W:",
#               pair.nhat_BA_W, "  p_WCa:",pair.p_WCa, "  p_WCb:",pair.p_WCb)
#         print("bodyA_index:",info.bodyA_index(),"  bodyB_index:", info.bodyB_index(),"  contact_force:", info.contact_force(),"  contact_point:",
#               info.contact_point(), "  separation_speed:",info.separation_speed(), "  slip_speed:",info.slip_speed())
#     print("########################")

sim_status 0 : Simulator successfully reached the boundary time (0.0).
q 0 : [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.00000000e-05 0.00000000e+00 3.34000019e-02]
v 0 : [ 0.        0.        0.        0.5       0.       -0.000981]
generalized_acc 0 : [ 0.          0.35593457  0.         -0.11728797  0.         -9.69271203]


sim_status 1 : Simulator successfully reached the boundary time (0.01).
q 1 : [9.99999989e-01 0.00000000e+00 1.49461991e-04 0.00000000e+00
 4.51173511e-03 0.00000000e+00 3.33929599e-02]
v 1 : [ 0.         -0.04737629  0.          0.39855985  0.          0.00334015]
generalized_acc 1 : [  0.         -23.15786999   0.          -5.5178945    0.
  -4.2921055 ]


sim_status 2 : Simulator successfully reached the boundary time (0.02).
q 2 : [9.99999991e-01 0.00000000e+00 1.38865958e-04 0.00000000e+00
 8.04460212e-03 0.00000000e+00 3.33836879e-02]
v 2 : [0.         0.04615959 0.         0.2991604  0.         0.0046396 ]
generalized_acc 2 : [  0.     

/home/kiwi/soft/virenv/pdrake/lib/python3.6/site-packages/pydrake/systems/meshcat_visualizer.py:604: RuntimeWarning: invalid value encountered in arcsin
  np.arcsin(np.linalg.norm(angle_axis)), angle_axis)


sim_status 20 : Simulator successfully reached the boundary time (0.2).
q 20 : [ 1.00000000e+00  0.00000000e+00 -1.19662702e-07  0.00000000e+00
  1.27369692e-02  0.00000000e+00  3.33750999e-02]
v 20 : [0.00000000e+00 2.59345095e-04 0.00000000e+00 8.59359301e-06
 0.00000000e+00 6.69802867e-09]
generalized_acc 20 : [0.00000000e+00 2.98508804e-02 0.00000000e+00 1.09097593e-03
 0.00000000e+00 2.25119933e-05]


sim_status 21 : Simulator successfully reached the boundary time (0.21).
q 21 : [ 1.00000000e+00  0.00000000e+00 -1.73081330e-07  0.00000000e+00
  1.27369662e-02  0.00000000e+00  3.33751000e-02]
v 21 : [ 0.00000000e+00 -1.00566750e-04  0.00000000e+00 -3.48657995e-06
  0.00000000e+00  4.57624546e-09]
generalized_acc 21 : [0.00000000e+00 7.77522962e-02 0.00000000e+00 2.56898762e-03
 0.00000000e+00 1.41953762e-05]


sim_status 22 : Simulator successfully reached the boundary time (0.22).
q 22 : [1.00000000e+00 0.00000000e+00 1.07206054e-07 0.00000000e+00
 1.27369849e-02 0.00000000e+00 3

sim_status 40 : Simulator successfully reached the boundary time (0.4).
q 40 : [1.00000000e+00 0.00000000e+00 2.04047075e-10 0.00000000e+00
 1.27369776e-02 0.00000000e+00 3.33751000e-02]
v 40 : [ 0.00000000e+00 -4.03338887e-08  0.00000000e+00 -1.20619105e-09
  0.00000000e+00 -3.49965364e-14]
generalized_acc 40 : [ 0.00000000e+00 -8.01135934e-05  0.00000000e+00 -2.69758337e-06
  0.00000000e+00  2.35976833e-11]


sim_status 41 : Simulator successfully reached the boundary time (0.41).
q 41 : [ 1.00000000e+00  0.00000000e+00 -6.95322626e-12  0.00000000e+00
  1.27369776e-02  0.00000000e+00  3.33751000e-02]
v 41 : [ 0.00000000e+00  1.29927784e-07  0.00000000e+00  4.34249288e-09
  0.00000000e+00 -2.16050193e-14]
generalized_acc 41 : [ 0.00000000e+00 -6.61401396e-06  0.00000000e+00 -1.75888236e-07
  0.00000000e+00 -7.29359905e-12]


sim_status 42 : Simulator successfully reached the boundary time (0.42).
q 42 : [ 1.00000000e+00  0.00000000e+00 -9.63910308e-11  0.00000000e+00
  1.27369776e-02 

sim_status 62 : Simulator successfully reached the boundary time (0.62).
q 62 : [1.00000000e+00 0.00000000e+00 2.04463453e-14 0.00000000e+00
 1.27369776e-02 0.00000000e+00 3.33751000e-02]
v 62 : [0.0000000e+00 5.7208372e-11 0.0000000e+00 1.9285568e-12 0.0000000e+00
 3.7072385e-15]
generalized_acc 62 : [ 0.00000000e+00 -1.24886407e-08  0.00000000e+00 -3.98104313e-10
  0.00000000e+00 -1.61531103e-21]


sim_status 63 : Simulator successfully reached the boundary time (0.63).
q 63 : [ 1.00000000e+00  0.00000000e+00 -4.67363921e-14  0.00000000e+00
  1.27369776e-02  0.00000000e+00  3.33751000e-02]
v 63 : [0.00000000e+00 6.68770253e-12 0.00000000e+00 1.90922520e-13
 0.00000000e+00 3.70697111e-15]
generalized_acc 63 : [ 0.00000000e+00  1.85362842e-08  0.00000000e+00  6.23258846e-10
  0.00000000e+00 -4.21753439e-21]


sim_status 64 : Simulator successfully reached the boundary time (0.64).
q 64 : [1.00000000e+00 0.00000000e+00 3.50428281e-15 0.00000000e+00
 1.27369776e-02 0.00000000e+00 3.33751

sim_status 84 : Simulator successfully reached the boundary time (0.84).
q 84 : [ 1.00000000e+00  0.00000000e+00 -9.97162925e-18  0.00000000e+00
  1.27369776e-02  0.00000000e+00  3.33751000e-02]
v 84 : [0.00000000e+00 1.45907889e-14 0.00000000e+00 4.88980829e-16
 0.00000000e+00 3.70643637e-15]
generalized_acc 84 : [ 0.00000000e+00 -1.05912191e-12  0.00000000e+00 -3.54807478e-14
  0.00000000e+00  0.00000000e+00]


sim_status 85 : Simulator successfully reached the boundary time (0.85).
q 85 : [1.00000000e+00 0.00000000e+00 6.07464650e-18 0.00000000e+00
 1.27369776e-02 0.00000000e+00 3.33751000e-02]
v 85 : [ 0.00000000e+00 -7.52481605e-15  0.00000000e+00 -2.52144498e-16
  0.00000000e+00  3.70643637e-15]
generalized_acc 85 : [0.00000000e+00 5.23790396e-13 0.00000000e+00 1.75749273e-14
 0.00000000e+00 0.00000000e+00]


sim_status 86 : Simulator successfully reached the boundary time (0.86).
q 86 : [ 1.00000000e+00  0.00000000e+00 -2.08528028e-17  0.00000000e+00
  1.27369776e-02  0.00000000